Setup and Imports

In [42]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os


Load and Preprocess Dataset

In [43]:
dataset_path = r"C:\Users\Acer\OneDrive\Documents\GenAI-HW-1\PetImages"

# Image parameters
img_size = (150, 150)
batch_size = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load train and validation sets
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Load test set (if available)
test_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 19968 images belonging to 2 classes.
Found 4991 images belonging to 2 classes.
Found 24959 images belonging to 2 classes.


 Define Models
1. Baseline Model (No Regularization)

In [44]:
def build_baseline_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


2. Model with L1/L2 Regularization

In [45]:
def build_l1_l2_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', kernel_regularizer=l1_l2(0.01, 0.01), input_shape=(150,150,3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu', kernel_regularizer=l1_l2(0.01, 0.01)),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu', kernel_regularizer=l1_l2(0.01, 0.01)),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(512, activation='relu', kernel_regularizer=l1_l2(0.01, 0.01)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


3. Model with Dropout

In [46]:
def build_dropout_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        MaxPooling2D(2,2),
        Dropout(0.2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.3),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dropout(0.4),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


4. Model with Data Augmentation

In [47]:
augmented_train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

augmented_train_generator = augmented_train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

def build_augmented_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


Found 19968 images belonging to 2 classes.


Train and Evaluate Models

In [48]:
models = {
    "Baseline": build_baseline_model(),
    "L1/L2 Regularization": build_l1_l2_model(),
    "Dropout": build_dropout_model(),
    "Data Augmentation": build_augmented_model()
}

history_dict = {}

for model_name, model in models.items():
    print(f"\nTraining {model_name} Model...\n")
    
    if model_name == "Data Augmentation":
        history = model.fit(augmented_train_generator, validation_data=val_generator, epochs=10)
    else:
        history = model.fit(train_generator, validation_data=val_generator, epochs=10)
    
    history_dict[model_name] = history



Training Baseline Model...



c:\Users\Acer\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
492/624 ━━━━━━━━━━━━━━━━━━━━ 1:08 519ms/step - accuracy: 0.5500 - loss: 0.7026

c:\Users\Acer\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


624/624 ━━━━━━━━━━━━━━━━━━━━ 355s 564ms/step - accuracy: 0.5634 - loss: 0.6916 - val_accuracy: 0.7469 - val_loss: 0.5065
Epoch 2/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 325s 520ms/step - accuracy: 0.7462 - loss: 0.5090 - val_accuracy: 0.7912 - val_loss: 0.4457
Epoch 3/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 343s 550ms/step - accuracy: 0.8205 - loss: 0.3971 - val_accuracy: 0.7672 - val_loss: 0.4915
Epoch 4/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 347s 557ms/step - accuracy: 0.8661 - loss: 0.3051 - val_accuracy: 0.8349 - val_loss: 0.3734
Epoch 5/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 414s 663ms/step - accuracy: 0.9263 - loss: 0.1837 - val_accuracy: 0.8271 - val_loss: 0.4668
Epoch 6/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 517s 829ms/step - accuracy: 0.9713 - loss: 0.0813 - val_accuracy: 0.8067 - val_loss: 0.7162
Epoch 7/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 681s 1s/step - accuracy: 0.9885 - loss: 0.0365 - val_accuracy: 0.8245 - val_loss: 0.7325
Epoch 8/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 415s 664ms/step - accuracy: 0.9917 - loss: 0.0269 

Compare Test Accuracies

In [49]:
test_results = {}

for model_name, model in models.items():
    print(f"\nEvaluating {model_name} Model...\n")
    test_loss, test_acc = model.evaluate(test_generator)
    test_results[model_name] = test_acc

print("\nTest Accuracies:")
for model, acc in test_results.items():
    print(f"{model}: {acc:.4f}")



Evaluating Baseline Model...

780/780 ━━━━━━━━━━━━━━━━━━━━ 164s 209ms/step - accuracy: 0.9424 - loss: 0.2691

Evaluating L1/L2 Regularization Model...

780/780 ━━━━━━━━━━━━━━━━━━━━ 190s 244ms/step - accuracy: 0.4986 - loss: 25.0427

Evaluating Dropout Model...

780/780 ━━━━━━━━━━━━━━━━━━━━ 174s 223ms/step - accuracy: 0.9336 - loss: 0.1817

Evaluating Data Augmentation Model...

780/780 ━━━━━━━━━━━━━━━━━━━━ 172s 221ms/step - accuracy: 0.8500 - loss: 0.3520

Test Accuracies:
Baseline: 0.9458
L1/L2 Regularization: 0.5004
Dropout: 0.9339
Data Augmentation: 0.8490
